<a href="https://colab.research.google.com/github/yachika-yashu/yachika-yashu/blob/main/Emotion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Notebook is structured as a linear, reproducible experiment:
# preprocessing → feature extraction → modeling → evaluation.

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer


import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, roc_auc_score


In [3]:
# Loading raw Twitter emotion data
# Dataset contains multiple emotions;
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv')

df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [4]:
# tweet_id does not contribute to prediction and is removed
df.drop(columns=['tweet_id'],inplace=True)
df

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
...,...,...
39995,neutral,@JohnLloydTaylor
39996,love,Happy Mothers Day All my love
39997,love,Happy Mother's Day to all the mommies out ther...
39998,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [5]:
# Restricting to happiness vs sadness to frame the task as a clean binary
# classification problem before attempting multi-class emotion modeling.
final_df = df[df['sentiment'].isin(['happiness','sadness'])]

In [6]:
final_df.sample(5)

,sentiment,content
15196,sadness,@Petronella I really want a Shish kebab... Goi...
24387,happiness,is almost done with the third/chapter cover pa...
17282,sadness,Dann and I are in Vegas. We misconnected and o...
30377,happiness,Great motorcycle ride along Hood Canal and thr...
27239,happiness,"is exhausted from a great weekend, and is look..."


In [7]:
final_df.shape

(10374, 2)

In [8]:

# Explicit label mapping avoids implicit assumptions about class ordering
final_df['sentiment'].replace({'happiness':1, 'sadness':0},inplace=True)

/tmp/ipython-input-966298388.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['sentiment'].replace({'happiness':1, 'sadness':0},inplace=True)
/tmp/ipython-input-966298388.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df['sentiment'].replace({'happiness':1, 'sadness':0},inplac

In [9]:
final_df.head()

,sentiment,content
1,0,Layin n bed with a headache ughhhh...waitin o...
2,0,Funeral ceremony...gloomy friday...
6,0,"I should be sleep, but im not! thinking about ..."
8,0,@charviray Charlene my love. I miss you
9,0,@kelcouch I'm sorry at least it's Friday?


In [10]:
train_data, test_data = train_test_split(final_df, test_size=0.2, random_state=42)

In [11]:
nltk.download('wordnet')
nltk.download('stopwords')

def lemmatization(text):
  # Lemmatization preserves meaning better than stemming
    lemmatizer= WordNetLemmatizer()

    text = text.split()

    text=[lemmatizer.lemmatize(y) for y in text]

    return " " .join(text)

def remove_stop_words(text):
    stop_words = set(stopwords.words("english"))
    Text=[i for i in str(text).split() if i not in stop_words]
    return " ".join(Text)

def removing_numbers(text):
    text=''.join([i for i in text if not i.isdigit()])
    return text

def lower_case(text):
   # Convert to lowercase for case-insensitive modeling

    text = text.split()

    text=[y.lower() for y in text]

    return " " .join(text)

def removing_punctuations(text):
     # Removing punctuation and digits reduces sparsity
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛',"", )

    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text =  " ".join(text.split())
    return text.strip()

def removing_urls(text):
   # URLs rarely contribute semantic meaning in sentiment tasks
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_small_sentences(df):
  # Very short texts tend to behave like noise in BoW space
    for i in range(len(df)):
        if len(df.text.iloc[i].split()) < 3:
            df.text.iloc[i] = np.nan

def normalize_text(df):
    df.content=df.content.apply(lambda content : lower_case(content))
    df.content=df.content.apply(lambda content : remove_stop_words(content))
    df.content=df.content.apply(lambda content : removing_numbers(content))
    df.content=df.content.apply(lambda content : removing_punctuations(content))
    df.content=df.content.apply(lambda content : removing_urls(content))
    df.content=df.content.apply(lambda content : lemmatization(content))
    return df

def normalized_sentence(sentence):
    sentence= lower_case(sentence)
    sentence= remove_stop_words(sentence)
    sentence= removing_numbers(sentence)
    sentence= removing_punctuations(sentence)
    sentence= removing_urls(sentence)
    sentence= lemmatization(sentence)
    return sentence

<>:34: SyntaxWarning: invalid escape sequence '\]'
<>:38: SyntaxWarning: invalid escape sequence '\s'
<>:34: SyntaxWarning: invalid escape sequence '\]'
<>:38: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-2179026750.py:34: SyntaxWarning: invalid escape sequence '\]'
  text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
/tmp/ipython-input-2179026750.py:38: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text)
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
train_data = normalize_text(train_data)
test_data = normalize_text(test_data)

In [13]:
train_data

,sentiment,content
23531,0,quot my problem miss you cause don t quot
8051,0,that s it done already one proof there s nothi...
11499,0,hungry food steal
31288,1,foot hurt finally bed will forget crunch over ...
18561,0,really ill atm
...,...,...
21697,1,chocolatesuze yes yes should especially wine m...
19445,0,kickzfadayz boy better get tonight
20216,1,tafe actually quite good
3258,0,minute boarding hour home window seat


In [14]:
X_train = train_data['content'].values
y_train = train_data['sentiment'].values

X_test = test_data['content'].values
y_test = test_data['sentiment'].values

In [15]:
# Apply Bag of Words (CountVectorizer)
# Bag-of-Words is used as a strong and interpretable baseline
# before experimenting with TF-IDF or embeddings
vectorizer = CountVectorizer()

# Fit only on training data to prevent data leakage
X_train_bow = vectorizer.fit_transform(X_train)

# Transform the test data using the same vectorizer
X_test_bow = vectorizer.transform(X_test)

In [16]:
train_df = pd.DataFrame(X_train_bow.toarray())

train_df['label'] = y_train
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,14223,14224,14225,14226,14227,14228,14229,14230,14231,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Define and train the XGBoost model
# XGBoost handles high-dimensional sparse features well
# and often outperforms linear models on noisy text data

xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
# Training on vectorized text features
xgb_model.fit(X_train_bow, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test_bow)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
# Classification report provides per-class performance breakdown
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [04:40:56] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.771566265060241
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.83      0.79      1060
           1       0.80      0.71      0.75      1015

    accuracy                           0.77      2075
   macro avg       0.77      0.77      0.77      2075
weighted avg       0.77      0.77      0.77      2075



In [18]:
# Make predictions
# Accuracy alone can be misleading,
# so multiple complementary metrics are reported
y_pred = xgb_model.predict(X_test_bow)
y_pred_proba = xgb_model.predict_proba(X_test_bow)[:, 1]

# Calculate evaluation metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

In [19]:
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"AUC: {auc}")

Precision: 0.7988950276243094
Recall: 0.7123152709359606
AUC: 0.8595775629705363


In [ ]:
# Future improvements:
# - Compare against TF-IDF and Logistic Regression baseline
# - Perform cross-validated hyperparameter tuning
# - Extend to multi-class emotion classification